In [1]:
!pip install transformers
!pip install torch


# 1. Prepare data

## 1.1 download data

In [2]:
import pandas as pd

# Define the paths to the local files
train_file = "./data/train.csv"
validation_file = "./data/dev.csv"
test_file = "./data/test.csv"

# Load the datasets
train_df = pd.read_csv(train_file)
validation_df = pd.read_csv(validation_file)
test_df = pd.read_csv(test_file)

# Display the columns in each DataFrame
print("Train columns:", train_df.columns)
print("Validation columns:", validation_df.columns)
print("Test columns:", test_df.columns)

# Display the first few rows to understand the structure
print("Train data preview:")
print(train_df.head())

print("Validation data preview:")
print(validation_df.head())

print("Test data preview:")
print(test_df.head())

Train columns: Index(['annotator_labels', 'genre', 'gold_label', 'pairID', 'promptID',
       'sentence1', 'translation1', 'sentence2', 'translation2'],
      dtype='object')
Validation columns: Index(['annotator_labels', 'genre', 'gold_label', 'pairID', 'promptID',
       'sentence1', 'translation1', 'sentence2', 'translation2'],
      dtype='object')
Test columns: Index(['annotator_labels', 'genre', 'gold_label', 'pairID', 'promptID',
       'sentence1', 'translation1', 'sentence2', 'translation2'],
      dtype='object')
Train data preview:
    annotator_labels    genre     gold_label   pairID  promptID  \
0     ['entailment']  fiction     entailment  134793e    134793   
1     ['entailment']  fiction     entailment   37397e     37397   
2     ['entailment']    slate     entailment    1069e      1069   
3  ['contradiction']    slate  contradiction   23505c     23505   
4  ['contradiction']   travel  contradiction   60529c     60529   

                                           sente

## 2. Define a function to preprocess the data:



In [ ]:
# import pandas as pd
# from transformers import AutoTokenizer
# import torch

# # Load the tokenizer
# model_name = "bert-base-multilingual-cased"  # Change this to AlephBERT or DictaBERT as needed
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# # Define the paths to the local files
# train_file = "/Users/user/Library/Mobile Documents/com~apple~CloudDocs/2 24/NLP/final proj/data/train.csv"
# validation_file = "/Users/user/Library/Mobile Documents/com~apple~CloudDocs/2 24/NLP/final proj/data/dev.csv"
# test_file = "/Users/user/Library/Mobile Documents/com~apple~CloudDocs/2 24/NLP/final proj/data/test.csv"

# # Load the datasets
# train_df = pd.read_csv(train_file)
# validation_df = pd.read_csv(validation_file)
# test_df = pd.read_csv(test_file)

# # Display the columns in each DataFrame
# print("Train columns:", train_df.columns)
# print("Validation columns:", validation_df.columns)
# print("Test columns:", test_df.columns)

# # Display the first few rows to understand the structure
# print("Train data preview:")
# print(train_df.head())

# print("Validation data preview:")
# print(validation_df.head())

# print("Test data preview:")
# print(test_df.head())

# # Adjust the preprocessing function based on actual column names
# def preprocess_data(df, tokenizer, max_length=128):
#     sentence1_col = 'sentence1'
#     sentence2_col = 'sentence2'
#     label_col = 'gold_label'
    
#     inputs = tokenizer(
#         df[sentence1_col].tolist(),
#         df[sentence2_col].tolist(),
#         padding=True,
#         truncation=True,
#         max_length=max_length,
#         return_tensors="pt"
#     )
    
#     label_mapping = {'contradiction': 0, 'entailment': 1, 'neutral': 2}
#     labels = df[label_col].map(label_mapping).tolist()
    
#     return inputs, torch.tensor(labels)

# # Preprocess the data
# train_inputs, train_labels = preprocess_data(train_df, tokenizer)
# val_inputs, val_labels = preprocess_data(validation_df, tokenizer)
# test_inputs, test_labels = preprocess_data(test_df, tokenizer)

# # Display the processed data
# print("Train inputs:", train_inputs)
# print("Train labels:", train_labels)
# print("Validation inputs:", val_inputs)
# print("Validation labels:", val_labels)
# print("Test inputs:", test_inputs)
# print("Test labels:", test_labels)


In [3]:
#this one
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

# Disable parallelism warning for tokenizers
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Load the tokenizer and model
model_name = "bert-base-multilingual-cased"  # Change this to AlephBERT or DictaBERT as needed
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Define the paths to the local files
# train_file = "/Users/user/Library/Mobile Documents/com~apple~CloudDocs/2 24/NLP/final proj/data/train.csv"
# validation_file = "/Users/user/Library/Mobile Documents/com~apple~CloudDocs/2 24/NLP/final proj/data/dev.csv"
# test_file = "/Users/user/Library/Mobile Documents/com~apple~CloudDocs/2 24/NLP/final proj/data/test.csv"

# Load the datasets
train_df = pd.read_csv(train_file)
validation_df = pd.read_csv(validation_file)
test_df = pd.read_csv(test_file)

# Display the columns in each DataFrame
print("Train columns:", train_df.columns)
print("Validation columns:", validation_df.columns)
print("Test columns:", test_df.columns)

# Display the first few rows to understand the structure
print("Train data preview:")
print(train_df.head())

print("Validation data preview:")
print(validation_df.head())

print("Test data preview:")
print(test_df.head())

# Adjust the preprocessing function based on actual column names
def preprocess_data(df, tokenizer, max_length=128):
    sentence1_col = 'sentence1'
    sentence2_col = 'sentence2'
    label_col = 'gold_label'
    
    inputs = tokenizer(
        df[sentence1_col].tolist(),
        df[sentence2_col].tolist(),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    
    label_mapping = {'contradiction': 0, 'entailment': 1, 'neutral': 2}
    labels = df[label_col].map(label_mapping).tolist()
    
    return inputs, torch.tensor(labels)

# Preprocess the data
train_inputs, train_labels = preprocess_data(train_df, tokenizer)
val_inputs, val_labels = preprocess_data(validation_df, tokenizer)
test_inputs, test_labels = preprocess_data(test_df, tokenizer)


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Train columns: Index(['annotator_labels', 'genre', 'gold_label', 'pairID', 'promptID',
       'sentence1', 'translation1', 'sentence2', 'translation2'],
      dtype='object')
Validation columns: Index(['annotator_labels', 'genre', 'gold_label', 'pairID', 'promptID',
       'sentence1', 'translation1', 'sentence2', 'translation2'],
      dtype='object')
Test columns: Index(['annotator_labels', 'genre', 'gold_label', 'pairID', 'promptID',
       'sentence1', 'translation1', 'sentence2', 'translation2'],
      dtype='object')
Train data preview:
    annotator_labels    genre     gold_label   pairID  promptID  \
0     ['entailment']  fiction     entailment  134793e    134793   
1     ['entailment']  fiction     entailment   37397e     37397   
2     ['entailment']    slate     entailment    1069e      1069   
3  ['contradiction']    slate  contradiction   23505c     23505   
4  ['contradiction']   travel  contradiction   60529c     60529   

                                           sente

## 3.Data loader



In [11]:
# Create DataLoader
def create_data_loader(inputs, labels, batch_size=16):
    dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], inputs['token_type_ids'], labels)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return data_loader

train_loader = create_data_loader(train_inputs, train_labels)
val_loader = create_data_loader(val_inputs, val_labels)
test_loader = create_data_loader(test_inputs, test_labels)


#subset part:
# # Create DataLoader
# def create_data_loader(inputs, labels, batch_size=16):
#     dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], inputs['token_type_ids'], labels)
#     data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
#     return data_loader

# # Use a subset of the training data
# subset_size = 100  # Use 100 samples for quick testing
# subset_train_dataset = TensorDataset(train_inputs['input_ids'][:subset_size], 
#                                      train_inputs['attention_mask'][:subset_size], 
#                                      train_inputs['token_type_ids'][:subset_size], 
#                                      train_labels[:subset_size])
# subset_train_loader = DataLoader(subset_train_dataset, batch_size=16, shuffle=True)

# # Validation and Test DataLoader
# val_loader = create_data_loader(val_inputs, val_labels)
# test_loader = create_data_loader(test_inputs, test_labels)


## 4. training setup

In [12]:
# Check for MPS and CUDA availability
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("MPS is available")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available")
else:
    device = torch.device('cpu')
    print("Using CPU")

# Training setup
model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = CrossEntropyLoss()



MPS is available


/Users/hadare/Documents/CodingProjects/SBERT/sbert/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## training loop

In [13]:
# # Training loop on a subset of the data
# epochs = 2  # Run only for 1 epoch for quick testing
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in tqdm(subset_train_loader):
#         input_ids, attention_mask, token_type_ids, labels = [b.to(device) for b in batch]
#         optimizer.zero_grad()
#         outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
#         logits = outputs.logits  # Assuming model returns logits as the first output
#         loss = loss_fn(logits, labels)
#         total_loss += loss.item()
#         loss.backward()
#         optimizer.step()
#     avg_train_loss = total_loss / len(subset_train_loader)
#     print(f'Epoch {epoch + 1}, Loss: {avg_train_loss:.4f}')


In [14]:
# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
    
        input_ids, attention_mask, token_type_ids, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        logits = outputs.logits  # Assuming model returns logits as the first output
        loss = loss_fn(logits, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}, Loss: {avg_train_loss:.4f}')


100%|██████████| 18332/18332 [2:41:52<00:00,  1.89it/s]   


Epoch 1, Loss: 0.6040


100%|██████████| 18332/18332 [2:06:13<00:00,  2.42it/s] 


Epoch 2, Loss: 0.4573


100%|██████████| 18332/18332 [2:18:01<00:00,  2.21it/s]  

Epoch 3, Loss: 0.3649


## 5. save the model

In [15]:
# Save the model
model.save_pretrained('./trained_sbert_model')
tokenizer.save_pretrained('./trained_sbert_model')


('./trained_sbert_model/tokenizer_config.json',
 './trained_sbert_model/special_tokens_map.json',
 './trained_sbert_model/vocab.txt',
 './trained_sbert_model/added_tokens.json',
 './trained_sbert_model/tokenizer.json')

## 6. Evaluation test set

In [16]:
# Evaluation on test set
model.eval()
correct_predictions = 0
total_predictions = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, token_type_ids, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)
accuracy = correct_predictions / total_predictions
print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.8042
